In [78]:


import pandas as pd
import numpy as np
import random
from sklearn.metrics.pairwise import cosine_similarity


from tqdm import tqdm

In [79]:
df_image_embeddings = pd.read_parquet('Ekstra_Bladet_image_embeddings/image_embeddings.parquet')

df_image_embeddings.head(2)

,article_id,image_embedding
0,9734738,"[-0.023272939, -0.03915197, -0.022547087, 0.04..."
1,8647636,"[-0.04257018, -0.038090054, -0.0064549753, 0.0..."


In [80]:
df = pd.read_parquet('ebnerd_small/train/behaviors.parquet')

df['predicted_article_id'] = df['article_ids_clicked'].apply(lambda ids: ids[0])
df_exploded = df.explode('article_ids_inview')

# Add a column to indicate if the article was recommended
df_exploded['rating'] = (df_exploded['predicted_article_id'] == df_exploded['article_ids_inview']).astype(int)

# def find_mode(series):
#     return mode(series).mode
#
# mode_ratings = df_exploded.groupby(['user_id', 'article_ids_inview'])['rating'].agg(find_mode).reset_index()

# Rename columns for clarity
df_exploded = df_exploded[['user_id', 'article_ids_inview', 'rating']]
df_exploded.columns = ['user_id', 'article_id', 'rating']
df_exploded.head(5)

,user_id,article_id,rating
0,139836,9778623,0
0,139836,9778682,0
0,139836,9778669,0
0,139836,9778657,1
0,139836,9778736,0


In [81]:
df_val = pd.read_parquet('ebnerd_small/validation/behaviors.parquet')
df_val.head(2)

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,96791,NaN,2023-05-28 04:21:24,9.0,NaN,2,"[9783865, 9784591, 9784679, 9784696, 9784710]",[9784696],22548,False,NaN,NaN,NaN,False,142,72.0,100.0
1,96798,NaN,2023-05-28 04:31:48,46.0,NaN,2,"[9782884, 9783865, 9782726, 9695098, 9782806, ...",[9784281],22548,False,NaN,NaN,NaN,False,143,16.0,28.0


In [ ]:
guessed_right = 0
guessed_wrong = 0
nothing_predicted_but_right = 0
nothing_predicted_but_wrong = 0
all_embeddings = np.vstack(df_image_embeddings['image_embedding'].values)
for i, row in tqdm(df_val.iterrows()):
    user = row["user_id"]
    inview = row["article_ids_inview"]
    actuals = row["article_ids_clicked"]
    df_user_filtered = df_exploded[df_exploded['user_id'] == user]
    df_user_likes = pd.merge(df_user_filtered, df_image_embeddings, on='article_id')
    df_cur_image_embeddings = df_image_embeddings[df_image_embeddings['article_id'].isin(inview)]
    if len(df_user_likes) == 0 or len(df_cur_image_embeddings) == 0:
        if random.choice(inview) in actuals:
            nothing_predicted_but_right += 1
        else:
            nothing_predicted_but_wrong += 1
    else:
        weighted_embeddings = np.vstack(df_user_likes['image_embedding'].values) * df_user_likes['rating'].values[:, np.newaxis]
        user_preference_embedding = np.mean(weighted_embeddings, axis=0)
        curr_all_embedings = np.vstack(df_cur_image_embeddings['image_embedding'].values)
        similarity_scores = cosine_similarity([user_preference_embedding], curr_all_embedings)[0]
        df_cur_image_embeddingsc = df_cur_image_embeddings[['article_id', 'image_embedding']].copy()
        df_cur_image_embeddingsc.loc[:, 'similarity_score'] = similarity_scores
        df_recommendations = df_cur_image_embeddingsc[~df_cur_image_embeddingsc['article_id'].isin(df_user_filtered['article_id'])]

        # Sort by similarity score
        df_recommendations = df_recommendations.sort_values(by='similarity_score', ascending=False)
        df_remaining = df_recommendations[df_recommendations['article_id'].isin(inview)]
        if len(df_remaining) > 0 and df_remaining.iloc[0]['article_id'] in actuals:
            guessed_right += 1
        elif len(df_remaining) > 0:
            guessed_wrong += 1
        else:
            if random.choice(inview) in actuals:
                nothing_predicted_but_right += 1
            else:
                nothing_predicted_but_wrong += 1
    if i % 20000 == 19999:
        print(f"Right 100%: {guessed_right}, by luck {nothing_predicted_but_right}, overall: {guessed_right + nothing_predicted_but_right}")
        print(f"Wrong 100%: {guessed_wrong}, by luck {nothing_predicted_but_wrong}, overall: {guessed_wrong + nothing_predicted_but_wrong}")
        print(f"Nothing predicted: {nothing_predicted_but_right + nothing_predicted_but_wrong}")
        print(f"The answer: {(guessed_right + nothing_predicted_but_right) / (nothing_predicted_but_right + nothing_predicted_but_wrong + guessed_right + guessed_wrong)}")

print(f"Right 100%: {guessed_right}, by luck {nothing_predicted_but_right}, overall: {guessed_right + nothing_predicted_but_right}")
print(f"Wrong 100%: {guessed_wrong}, by luck {nothing_predicted_but_wrong}, overall: {guessed_wrong + nothing_predicted_but_wrong}")
print(f"Nothing predicted: {nothing_predicted_but_right + nothing_predicted_but_wrong}")
print(f"The answer: {(guessed_right + nothing_predicted_but_right) / (nothing_predicted_but_right + nothing_predicted_but_wrong + guessed_right + guessed_wrong)}")

20001it [30:40,  5.79it/s]

Right 100%: 2333, by luck 423, overall: 2756
Wrong 100%: 14323, by luck 2921, overall: 17244
Nothing predicted: 3344
The answer: 0.1378


40001it [54:22, 12.25it/s]

Right 100%: 4561, by luck 876, overall: 5437
Wrong 100%: 28520, by luck 6043, overall: 34563
Nothing predicted: 6919
The answer: 0.135925


60006it [1:11:20, 46.53it/s]

Right 100%: 6849, by luck 1295, overall: 8144
Wrong 100%: 42796, by luck 9060, overall: 51856
Nothing predicted: 10355
The answer: 0.13573333333333334


80007it [1:33:02, 40.27it/s] 

Right 100%: 9171, by luck 1706, overall: 10877
Wrong 100%: 57110, by luck 12013, overall: 69123
Nothing predicted: 13719
The answer: 0.1359625


90575it [1:43:59, 14.52it/s]


KeyboardInterrupt: 